In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# COMPAS Bias Detection Analysis\n",
    "## AI Ethics Assignment - Fairness Metrics and Bias Analysis\n",
    "\n",
    "This notebook performs comprehensive fairness analysis on the COMPAS dataset, calculating key metrics to detect racial bias."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Import Libraries and Load Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.metrics import confusion_matrix, classification_report\n",
    "from pathlib import Path\n",
    "\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)\n",
    "\n",
    "# Load dataset\n",
    "csv_path = Path('data/compas_raw/compas.csv')\n",
    "df = pd.read_csv(csv_path)\n",
    "\n",
    "print(f'✓ Dataset loaded: {len(df):,} rows, {len(df.columns)} columns')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Prepare Data for Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Handle column name variation\n",
    "if 'two_year_recidivism' in df.columns:\n",
    "    recid_col = 'two_year_recidivism'\n",
    "elif 'two_year_recid' in df.columns:\n",
    "    recid_col = 'two_year_recid'\n",
    "    df['two_year_recidivism'] = df['two_year_recid']\n",
    "else:\n",
    "    print('Error: No recidivism column found')\n",
    "\n",
    "# Create high-risk binary prediction\n",
    "df['high_risk'] = (df['decile_score'] >= 5).astype(int)\n",
    "\n",
    "# Filter to major racial groups\n",
    "df_analysis = df[df['race'].isin(['African-American', 'Caucasian'])].copy()\n",
    "\n",
    "print(f'✓ Data prepared:')\n",
    "print(f'  Total records: {len(df_analysis):,}')\n",
    "print(f'  African-American: {len(df_analysis[df_analysis[\"race\"]==\"African-American\"]):,}')\n",
    "print(f'  Caucasian: {len(df_analysis[df_analysis[\"race\"]==\"Caucasian\"]):,}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Calculate Fairness Metrics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def calculate_metrics(y_true, y_pred):\n",
    "    \"\"\"Calculate fairness metrics\"\"\"\n",
    "    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()\n",
    "    \n",
    "    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0\n",
    "    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0\n",
    "    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0\n",
    "    accuracy = (tp + tn) / (tp + tn + fp + fn)\n",
    "    \n",
    "    return {\n",
    "        'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp,\n",
    "        'fpr': fpr, 'fnr': fnr, 'tpr': tpr, 'accuracy': accuracy\n",
    "    }\n",
    "\n",
    "# Calculate metrics for each race\n",
    "metrics = {}\n",
    "\n",
    "for race in ['African-American', 'Caucasian']:\n",
    "    race_df = df_analysis[df_analysis['race'] == race]\n",
    "    y_true = race_df['two_year_recidivism'].values\n",
    "    y_pred = race_df['high_risk'].values\n",
    "    \n",
    "    metrics[race] = calculate_metrics(y_true, y_pred)\n",
    "\n",
    "# Display metrics\n",
    "print('FAIRNESS METRICS BY RACE')\n",
    "print('='*70)\n",
    "\n",
    "for race in ['African-American', 'Caucasian']:\n",
    "    m = metrics[race]\n",
    "    print(f'\\n{race}:')\n",
    "    print(f'  False Positive Rate (FPR): {m[\"fpr\"]:.4f} ({m[\"fpr\"]*100:.2f}%)')\n",
    "    print(f'  False Negative Rate (FNR): {m[\"fnr\"]:.4f} ({m[\"fnr\"]*100:.2f}%)')\n",
    "    print(f'  True Positive Rate (TPR): {m[\"tpr\"]:.4f} ({m[\"tpr\"]*100:.2f}%)')\n",
    "    print(f'  Accuracy: {m[\"accuracy\"]:.4f}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Disparate Impact Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate disparate impact ratio\n",
    "fpr_aa = metrics['African-American']['fpr']\n",
    "fpr_c = metrics['Caucasian']['fpr']\n",
    "\n",
    "di_ratio = fpr_aa / fpr_c if fpr_c > 0 else float('inf')\n",
    "\n",
    "print('DISPARATE IMPACT ANALYSIS')\n",
    "print('='*70)\n",
    "print(f'\\nFalse Positive Rate (African-American): {fpr_aa:.4f} ({fpr_aa*100:.2f}%)')\n",
    "print(f'False Positive Rate (Caucasian): {fpr_c:.4f} ({fpr_c*100:.2f}%)')\n",
    "print(f'\\nDisparate Impact Ratio: {di_ratio:.4f}')\n",
    "print(f'\\nInterpretation:')\n",
    "print(f'African-American defendants are flagged as high-risk at {di_ratio:.2f}x')\n",
    "print(f'the rate of Caucasian defendants.')\n",
    "\n",
    "# EEOC 80% rule\n",
    "print(f'\\nEEOC 80% Rule Standard:')\n",
    "print(f'  Target: Ratio >= 0.8')\n",
    "print(f'  Current: {di_ratio:.4f}')\n",
    "if di_ratio >= 0.8:\n",
    "    print(f'  Status: ✓ PASSES (acceptable)')\n",
    "else:\n",
    "    print(f'  Status: ⚠ FAILS (evidence of discrimination)')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Visualization: Key Metrics Comparison"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig, axes = plt.subplots(2, 2, figsize=(14, 10))\n",
    "fig.suptitle('COMPAS Fairness Metrics by Race', fontsize=16, fontweight='bold')\n",
    "\n",
    "races = ['African-American', 'Caucasian']\n",
    "colors = ['#FF6B6B', '#4ECDC4']\n",
    "\n",
    "# Plot 1: FPR Comparison\n",
    "fprs = [metrics[r]['fpr'] for r in races]\n",
    "axes[0, 0].bar(races, fprs, color=colors, alpha=0.7, edgecolor='black', linewidth=2)\n",
    "axes[0, 0].set_ylabel('Rate', fontweight='bold')\n",
    "axes[0, 0].set_title('False Positive Rate (FPR)', fontweight='bold')\n",
    "for i, v in enumerate(fprs):\n",
    "    axes[0, 0].text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')\n",
    "\n",
    "# Plot 2: FNR Comparison\n",
    "fnrs = [metrics[r]['fnr'] for r in races]\n",
    "axes[0, 1].bar(races, fnrs, color=colors, alpha=0.7, edgecolor='black', linewidth=2)\n",
    "axes[0, 1].set_ylabel('Rate', fontweight='bold')\n",
    "axes[0, 1].set_title('False Negative Rate (FNR)', fontweight='bold')\n",
    "for i, v in enumerate(fnrs):\n",
    "    axes[0, 1].text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')\n",
    "\n",
    "# Plot 3: Accuracy Comparison\n",
    "accs = [metrics[r]['accuracy'] for r in races]\n",
    "axes[1, 0].bar(races, accs, color=colors, alpha=0.7, edgecolor='black', linewidth=2)\n",
    "axes[1, 0].set_ylabel('Accuracy', fontweight='bold')\n",
    "axes[1, 0].set_title('Overall Accuracy', fontweight='bold')\n",
    "axes[1, 0].set_ylim([0, 1])\n",
    "for i, v in enumerate(accs):\n",
    "    axes[1, 0].text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')\n",
    "\n",
    "# Plot 4: Disparate Impact Ratio\n",
    "axes[1, 1].barh(['Disparate Impact\\nRatio'], [di_ratio], color='#95E1D3', alpha=0.7, edgecolor='black', linewidth=2)\n",
    "axes[1, 1].axvline(x=0.8, color='red', linestyle='--', linewidth=2, label='EEOC 80% threshold')\n",
    "axes[1, 1].set_xlabel('Ratio', fontweight='bold')\n",
    "axes[1, 1].set_title('Disparate Impact Ratio (FPR)', fontweight='bold')\n",
    "axes[1, 1].text(di_ratio + 0.05, 0, f'{di_ratio:.2f}x', va='center', fontweight='bold', fontsize=12)\n",
    "axes[1, 1].legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Confusion Matrices"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig, axes = plt.subplots(1, 2, figsize=(14, 5))\n",
    "fig.suptitle('Confusion Matrices by Race', fontsize=14, fontweight='bold')\n",
    "\n",
    "for idx, race in enumerate(['African-American', 'Caucasian']):\n",
    "    m = metrics[race]\n",
    "    cm = np.array([[m['tn'], m['fp']], [m['fn'], m['tp']]])\n",
    "    \n",
    "    sns.heatmap(cm, annot=True, fmt='d', cmap=['Reds', 'Blues'][idx],\n",
    "                ax=axes[idx], cbar=False,\n",
    "                xticklabels=['Predicted Negative', 'Predicted Positive'],\n",
    "                yticklabels=['Actual Negative', 'Actual Positive'])\n",
    "    axes[idx].set_title(f'{race}', fontweight='bold')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Key Findings Summary"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print('\\n' + '='*70)\n",
    "print('KEY FINDINGS - RACIAL BIAS IN COMPAS')\n",
    "print('='*70)\n",
    "\n",
    "print(f'\\n1. PRIMARY FINDING - Disparate Impact:')\n",
    "print(f'   African-American defendants are flagged as high-risk at {di_ratio:.2f}x')\n",
    "print(f'   the rate of Caucasian defendants.')\n",
    "\n",
    "print(f'\\n2. FALSE POSITIVE RATE (FPR) - Critical Metric:')\n",
    "print(f'   African-American: {fpr_aa*100:.2f}%')\n",
    "print(f'   Caucasian: {fpr_c*100:.2f}%')\n",
    "print(f'   Difference: {abs(fpr_aa - fpr_c)*100:.2f} percentage points')\n",
    "print(f'\\n   Interpretation: Among people who DO NOT reoffend,')\n",
    "print(f'   {fpr_aa*100:.1f}% of African-Americans are incorrectly flagged as')\n",
    "print(f'   high-risk, vs only {fpr_c*100:.1f}% of Caucasians.')\n",
    "\n",
    "print(f'\\n3. LEGAL STANDARD - EEOC 80% Rule:')\n",
    "print(f'   Current Ratio: {di_ratio:.4f}')\n",
    "print(f'   Acceptable Threshold: >= 0.8')\n",
    "print(f'   Status: {\"✓ PASSES\" if di_ratio >= 0.8 else \"✗ FAILS\"}')\n",
    "\n",
    "print(f'\\n4. SYSTEMIC BIAS IMPLICATIONS:')\n",
    "print(f'   - African-American defendants receive harsher treatment')\n",
    "print(f'   - Bias compounds through criminal justice system')\n",
    "print(f'   - Creates self-fulfilling prophecy in sentencing')\n",
    "print(f'   - Perpetuates systemic racial inequality')\n",
    "\n",
    "print(f'\\n5. ETHICAL ASSESSMENT:')\n",
    "print(f'   This system VIOLATES fairness principles:')\n",
    "print(f'   - Justice: Unequal outcomes for same behavior')\n",
    "print(f'   - Non-maleficence: Causes demonstrable harm')\n",
    "print(f'   - Autonomy: Biased predictions limit opportunities')\n",
    "\n",
    "print(f'\\n' + '='*70)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Recommendations"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print('RECOMMENDED REMEDIATION STEPS')\n",
    "print('='*70)\n",
    "\n",
    "print('\\nIMMEDIATE (0-3 months):')\n",
    "print('  1. Stop using COMPAS as sole decision-making tool')\n",
    "print('  2. Implement mandatory human review for all high-risk classifications')\n",
    "print('  3. Audit all prior sentences influenced by biased scores')\n",
    "\n",
    "print('\\nSHORT-TERM (3-6 months):')\n",
    "print('  1. Retrain model with explicit fairness constraints')\n",
    "print('  2. Remove proxy variables correlated with race')\n",
    "print('  3. Achieve disparate impact ratio ≥ 0.85')\n",
    "\n",
    "print('\\nLONG-TERM (6+ months):')\n",
    "print('  1. Develop alternative risk assessment methods')\n",
    "print('  2. Involve affected communities in system redesign')\n",
    "print('  3. Establish independent algorithmic audit board')\n",
    "\n",
    "print('\\n' + '='*70)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

NameError: name 'null' is not defined